# Python --> try with stochastic target to emulate real life

In [1]:
import importlib
import numpy as np
import time

In [144]:
import dna_v2

In [145]:
from dna_v2 import DNA # To get this to work, needed to put 'if name == main' at bottom of dna.py 

In [146]:
importlib.reload(dna_v2)

<module 'dna_v2' from '/home/mjmrose/workspace/sbox-mjmrose/Bids/genAlgo/dna_v2.py'>

# Testing

In [35]:
tgt = "To be or not to be."
pop_sz = 8
mut_rate = 0.3
fitness_exp = 2
midpt_bool = False
pop = Population(tgt, mut_rate, pop_sz, fitness_exp)

In [39]:
draw(pop, 2)

World Record: 0.05263157894736842
Best: Vir'O]oH$)$!C->i)-m
Average: 0.019736842105263157


In [40]:
pop.mating_pool

[(5, 0.3333333333333333),
 (6, 0.3333333333333333),
 (7, 0.3333333333333333),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0)]

In [41]:
for i in range(len(pop.population)):
    print(pop.population[i].genes, pop.population[i].fitness)

['d', '6', 'o', '9', 'W', 'Z', '1', '<', 'n', '1', 'c', '#', 'Y', '#', '5', 'I', 'D', 'd', '6'] 0.0
['\\', 's', 'z', ']', '4', 'N', '8', 'h', 'k', '/', 'f', '#', 'L', ' ', 'W', 'p', 'A', ',', 'u'] 0.0
['C', 'j', 'g', 'o', 'w', '&', 'f', 'B', 'Z', 'F', 'V', 'Y', 'K', 'h', '9', '6', 'A', 'Q', '!'] 0.0
['V', 'D', '=', 'z', 'O', ']', 'o', 'H', '$', ')', '`', '=', 'C', '-', ' ', 'i', ')', '-', 'H'] 0
['V', 'i', 'r', "'", 's', ']', 'y', 'H', '$', 'P', '$', 'x', '?', '-', '6', 'v', 'x', '-', 'j'] 0
['V', 'i', 'r', "'", 'O', ']', 'o', 'H', '$', ')', '$', '!', 'C', '-', '>', 'i', ')', '-', 'm'] 0.0027700831024930744
['V', 'i', 'r', "'", 'O', 'P', 'o', 'H', '$', 'R', '$', '!', 'C', '-', '>', 'i', ')', '-', 'm'] 0.0027700831024930744
['V', 'i', 'r', "'", 'O', 'p', 'o', 'H', '$', 'R', '(', 'X', '?', '0', '6', 'v', ')', '-', 'm'] 0.0027700831024930744


# Class Definition

In [150]:
# A class to describe a population of virtual organisms
# In this case, each organism is just an instance of a DNA object

class Population:
    def __init__(self, tgt, mut_rate, pop_sz, fitness_exp, replace_bool = True, midpt_bool = False, verbose=False, debug=False):

        self.population = [] # Array to hold the current population
        self.mating_pool = [] # List which we will use for our "mating pool"
        self.generations = 0 # Number of generations
        self.finished = False # Are we finished evolving?
        self.tgt = tgt # Target phrase
        self.mut_rate = mut_rate # Mutation rate
        self.perfect_score = 1
        self.best = ""
        self.fitness_sum = 0
        self.fitness_exp = fitness_exp
        self.replace_bool = replace_bool
        self.midpt_bool = midpt_bool
        self.verbose = verbose
        self.debug = debug

        for i in range(pop_sz):
            self.population.append( DNA( len(self.tgt), verbose=self.verbose ) )

    def calc_fitness(self):
        '''
        Calculates fitness for every member of the population, exponentiates the fitness and calculates population sum
        '''
        if self.debug: print("start of calc_fitness")
        self.fitness_sum = 0
    
        for i in range( len(self.population) ):
           
            self.population[i].calc_fitness(self.tgt)
            
            self.population[i].fitness = self.population[i].fitness**self.fitness_exp
            
            self.fitness_sum += self.population[i].fitness

    def gen_mating_pool(self):
        if self.debug: print("start of gen_mating_pool")
        self.mating_pool = []
    
        for i in range( len(self.population) ): 
    
            # Appending (idx, normalized fitness) for each mem of pop
            self.mating_pool.append( (i, self.population[i].fitness / max(self.fitness_sum, 1e-4) ) )
    
        # Sorting by fitness score in descending order
        self.mating_pool.sort(reverse=True, key = lambda x : x[1])
        
    
    def pick_mem_from_mating_pool(self):
        '''
        Selects a member (mem) from the mating pool to participate in crossover.

        Steps for selection:
            1). Draw random number b/w 0-1 (val)
            2). Subtract normalized fitness of 1st mem of mating pool (highest fitness) from val
            3). If val is now negative, mem of pop corresponding to first mem of mating pool is selected for crossover.
            4). If val is still positive, move to 2nd mem of mating pool and repeat until val is negative
        '''
        if self.debug: print("Start of pick_mem_from_mating_pool")
            
        val = np.random.random()
        
        if self.verbose: print(f"val: {val}")
        if self.verbose: print(f"mating pool: {self.mating_pool}")
            
        for i in range( len(self.mating_pool) ):
            val -= self.mating_pool[i][1]
            if val < 0:
                break
        if self.verbose: print(f"idx of mating pool: {self.mating_pool[i][0]}")
        return self.mating_pool[i][0]
    

    # Create a new generation -->  Refill the population with children from the mating pool
    def gen_new_pop(self, num_new_mems):
        '''
        Generates new members (mems) of population by probabilistically mating existing mems of the population.
        Mems of the population w/ higher fitness are more likely to be selected for mating.

        Inputs:
            num_new_mems: number of new mems to add to the population or to replace in existing population
            replace_bool: if True, will replace 'num_new_mems' mems of pop w/ lowest fitness w/ new children 
                          if False, new children will be appended to existing population
        '''
        if self.debug: print("Start of gen_new_pop")
        children = []
        
        for i in range( num_new_mems ): 
                
            idx1 = self.pick_mem_from_mating_pool()
            idx2 = self.pick_mem_from_mating_pool()
            
            #THIS WAS NEW --> would want with low mutation rate
            if idx1 == idx2: # Sampling a random (top 5 fitness) member if idx1 == idx2
                rand_hi_idx = np.random.randint( min(10, len(self.population)) )
                idx2 = self.mating_pool[rand_hi_idx][0]
                if self.verbose: print("idx1 == idx2 dealt with")
            
            partnerA = self.population[idx1]
            partnerB = self.population[idx2]
            
            if self.verbose: print(f"PartnerA: {partnerA.get_phrase()}")
            if self.verbose: print(f"PartnerB: {partnerB.get_phrase()}")
    
            child = partnerA.crossover(partnerB, midpt_bool=self.midpt_bool)
            child.mutate(self.mut_rate)
            if self.verbose: print(f"Child: {child.get_phrase()}")
                
            children.append(child)

        if self.replace_bool:    
            for i in range(len(children)):
                replace_idx = self.mating_pool[len(self.mating_pool) - i - 1][0]
                self.population[replace_idx] = children[i] # Overwrites fitness of self.population[replace_idx] w/ 0
        else:
            self.population.extend(children)

        self.generations += 1


    def get_best(self):
        return self.best

    def evaluate(self):
        '''
        Computes the current "most fit" member of the population and whether the perfect score has been achieved.
        '''
        world_record = 0
        idx = 0
        for i in range(len(self.population)): 
            if self.population[i].fitness > world_record:
                idx = i
                world_record = self.population[i].fitness
                
        print(f"World Record: {world_record**(1/self.fitness_exp)}")

        self.best = self.population[idx].get_phrase()
        if world_record**(1/self.fitness_exp) == self.perfect_score:
            self.finished = True

    def is_finished(self):
        return self.finished

    def get_generations(self):
        return self.generations

    # Compute average fitness for the population
    def get_average_fitness(self):
        total = 0
        for i in range( len( self.population ) ):
            total += self.population[i].fitness**(1/self.fitness_exp)
        return total / len(self.population)

In [151]:
def draw(population, num_new_mems):
    
    # Calculate fitness for each mem of pop, take fitness**fitness_exp and calc fitness sum
    population.calc_fitness()
    
    # Generate mating pool array by sorting normalized fitness values
    population.gen_mating_pool()
    
    # Generate new population mems by crossover b/w existing mems of mating pool
    # Either replace existing mems or add new mems to pop
    population.gen_new_pop(num_new_mems)
    
    # Compute most fit mem of pop and determine if finished
    population.evaluate()
    
    print(f"Best: {population.get_best()}")
    print(f"Average: {population.get_average_fitness()}")

    # If we found the target phrase, stop
    if population.is_finished():
        print("We did it :)")
        print(f"Result: {pop.get_best()}")
        print(f"Num gens: {pop.get_generations()}")

### Initializing a population

In [152]:
tgt = "To be or not to be."
pop_sz = 500
mut_rate = 0.3
fitness_exp = 2
pop = Population(tgt, mut_rate, pop_sz, fitness_exp)

# Testing speed of convergence w/ gene selection based on fitness

In [153]:
start = time.time()
for i in range(1200):
    draw(pop, 100)
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

World Record: 0.10526315789473684
Best: XFE.5\AVyjo\%P+#+ew
Average: 0.011473684210526308
World Record: 0.21052631578947367
Best: -ooz% hu\0oZE<G("*.
Average: 0.022105263157894697
World Record: 0.2631578947368421
Best: [oqz6.ju no1,=W0bKc
Average: 0.03842105263157894
World Record: 0.2631578947368421
Best: Toq1[kS9 Lo3,tK)MD3
Average: 0.058631578947368604
World Record: 0.2631578947368421
Best: 'oo1c huESoZEqGzb6.
Average: 0.07726315789473724
World Record: 0.2631578947368421
Best: -$osj hu ooZ sV(=,.
Average: 0.08989473684210557
World Record: 0.2631578947368421
Best: -$osj hu ooZ sV(=,.
Average: 0.10494736842105282
World Record: 0.3157894736842105
Best: ToWzmKOu no12MTSX,.
Average: 0.1166315789473685
World Record: 0.3157894736842105
Best: ToWzmKOu no12MTSX,.
Average: 0.12526315789473647
World Record: 0.3157894736842105
Best: ToWzmKOu no12MTSX,.
Average: 0.13357894736842044
World Record: 0.3157894736842105
Best: -o?<S hH ioZ <Z]"w.
Average: 0.14410526315789446
World Record: 0.368421052631

World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.36505263157894824
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.36621052631579043
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.36726315789473796
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.36842105263158004
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.36957894736842223
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.3710526315789485
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.3725263157894748
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.374105263157896
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.3767368421052645
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.3786315789473699
World Record: 0.6842105263157895
Best: 0o b  o. nvV t[ be.
Average: 0.3803157894736858
World Record: 0.6842105263157895
Best: 

World Record: 0.6842105263157895
Best: Tx bC^oy noc/to be.
Average: 0.46442105263157557
World Record: 0.6842105263157895
Best: [o gw o0 not Zo be3
Average: 0.46557894736841765
World Record: 0.6842105263157895
Best: [o gw o0 not Zo be3
Average: 0.4663157894736808
World Record: 0.6842105263157895
Best: [o gw o0 not Zo be3
Average: 0.46736842105262816
World Record: 0.6842105263157895
Best: [o gw o0 not Zo be3
Average: 0.4684210526315755
World Record: 0.7368421052631579
Best: no b\ 9r noH topbe.
Average: 0.4698947368421018
World Record: 0.7368421052631579
Best: no b\ 9r noH topbe.
Average: 0.4712631578947333
World Record: 0.7368421052631579
Best: no b\ 9r noH topbe.
Average: 0.47231578947368064
World Record: 0.7368421052631579
Best: no b\ 9r noH topbe.
Average: 0.4736842105263122
World Record: 0.7368421052631579
Best: no b\ 9r noH topbe.
Average: 0.4746315789473648
World Record: 0.7368421052631579
Best: To ;e ji noU to bM.
Average: 0.47547368421052266
World Record: 0.7368421052631579
Best:

World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5309473684210495
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5313684210526285
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5318947368421022
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5325263157894707
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5327368421052602
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5329473684210496
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5330526315789444
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5333684210526286
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.533578947368418
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.5337894736842076
World Record: 0.7894736842105263
Best: To 5e o. noE to Te.
Average: 0.533999999999997
World Record: 0.7894736842105263
Best: To 5e 

World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5675789473684203
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5682105263157887
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5684210526315782
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5684210526315782
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5684210526315782
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5686315789473676
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5688421052631571
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5688421052631571
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5691578947368414
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5692631578947362
World Record: 0.8421052631578947
Best: To be$or 2ot to be<
Average: 0.5694736842105257
World Record: 0.8421052631578947
Best: To b

World Record: 0.8947368421052632
Best: To be o( notzto be.
Average: 0.5975789473684212
World Record: 0.8947368421052632
Best: To be o( notzto be.
Average: 0.5977894736842106
World Record: 0.8947368421052632
Best: To be o( notzto be.
Average: 0.5980000000000001
World Record: 0.8947368421052632
Best: To be o( notzto be.
Average: 0.5981052631578949
World Record: 0.8947368421052632
Best: To be o( notzto be.
Average: 0.5981052631578949
World Record: 0.8947368421052632
Best: To be o( notzto be.
Average: 0.5982105263157896
World Record: 1.0
Best: To be or not to be.
Average: 0.5987368421052631
We did it :)
Result: To be or not to be.
Num gens: 491

Time: 8.626277208328247


# Testing midpt

In [394]:
tgt = "To be or not to be."
pop_sz = 500
mut_rate = 0.3
fitness_exp = 2
pop = Population(tgt, mut_rate, pop_sz, fitness_exp)

In [ ]:
start = time.time()
for i in range(1000):
    draw(pop, 100)
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

World Record: 0.10526315789473684
Best: ([&x"6C8 0ZSHWq tGP
Average: 0.01026315789473683
World Record: 0.15789473684210525
Best: /HBFC*<U xHf2\3 b_$
Average: 0.01661654135338342
World Record: 0.15789473684210525
Best: /HBFC*<U xHf2\3 b_$
Average: 0.022828947368421018
World Record: 0.21052631578947367
Best: Yv'bq&trgF4uppoabAS
Average: 0.030292397660818735
World Record: 0.21052631578947367
Best: Yv'bq&trgF4uppoabAS
Average: 0.036105263157894835
World Record: 0.2631578947368421
Best: WQ'bYA0l K4trtsDb(*
Average: 0.04124401913875615
World Record: 0.2631578947368421
Best: WQ'bYA0l K4trtsDb(*
Average: 0.0467543859649125
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.0517408906882594
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.05687969924812078
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.06063157894736904
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.06434210526315867
World Record: 0.3684210526

World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1401947368420947
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14062011464303267
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1408204334365218
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14092999489012717
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14124493927124412
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1414586466165303
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14169811320753592
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14187899655680125
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14214912280700612
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14239497827135497
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14272248803826598
World Record: 0.47368421

World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15502590673573458
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1551112316874494
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15527935222670386
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1554162191192098
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15555971146137773
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1556698564593133
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1557762496693828
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15593157894735157
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15598586017280333
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15611255862426662
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15626393570130545
World Record: 0.473684210

# Debugging

In [ ]:
draw(pop, 5, True)

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())



In [ ]:
pop.mating_pool

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())


In [ ]:

pop.mating_pool